In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path

data_root = 'data'

fast_moves_df = pd.read_json(f'{data_root}/pogoapi/fast_moves.json')
charged_moves_df = pd.read_json(f'{data_root}/pogoapi/charged_moves.json')

fast_moves_df.rename(columns={
	'name':'fast_name',
	'type': 'fast_type',
	'power': 'fast_power',
	'energy_delta': 'fast_energy'
}, inplace=True)

charged_moves_df.rename(columns={
	'name':'charged_name',
	'type': 'charged_type',
	'power': 'charged_power',
	'energy_delta': 'charged_energy'
}, inplace=True)

fast_moves_df = fast_moves_df[['fast_name', 'fast_type', 'fast_power', 'fast_energy']]
charged_moves_df = charged_moves_df[['charged_name', 'charged_type', 'charged_power', 'charged_energy']]

merged_moves_df = pd.merge(fast_moves_df, charged_moves_df, how='cross')

merged_moves_df['charged_rate'] = np.ceil(merged_moves_df['charged_energy'].abs() / merged_moves_df['fast_energy'])
merged_moves_df['charged_rate'] = merged_moves_df['charged_rate'].replace([np.inf, -np.inf], None)

merged_moves_df.sort_values(by=['charged_rate'], ascending=True, inplace=True)

Path(f'{data_root}/csv').mkdir(parents=True, exist_ok=True)
merged_moves_df.to_csv(f'{data_root}/csv/merged_moves.csv',index=False)

display(merged_moves_df)
print('Merged move generated')